In [ ]:
from dcicutils import ff_utils
import json

env = 'data'
keys = open('/Users/koray/keypairs.json', 'r').read()
my_key = json.loads(keys)['koray_data']

intr = "intermediate file"

Converter = {
    "bwa-mem 0.2.5":{"bam":intr},
    "hi-c-processing-bam 0.2.5":{"bam": "alignment",
                                 "pairs": 'contact list-replicate'},
    "hi-c-processing-pairs 0.2.5":{"normvector_juicerformat": "juicebox norm vector",
                                   "hic": "contact matrix",
                                   "mcool":"contact matrix",
                                   "pairs":"contact list-combined"},
    "hi-c-processing-pairs-nore 0.2.5":{"normvector_juicerformat": "juicebox norm vector",
                                   "hic": "contact matrix",
                                   "mcool":"contact matrix",
                                   "pairs":"contact list-combined"},
    "hi-c-processing-pairs-nonorm 0.2.5":{
                                   "hic": "contact matrix",
                                   "mcool":"contact matrix",
                                   "pairs":"contact list-combined"},
    "hi-c-processing-pairs-nore-nonorm 0.2.5":{
                                   "hic": "contact matrix",
                                   "mcool":"contact matrix",
                                   "pairs":"contact list-combined"}
    
}

my_url = '/search/?file_type=No%20value&type=FileProcessed&limit=all'
all_files = ff_utils.search_metadata(my_url, key=my_key)
print(len(all_files))

In [ ]:
my_wfrs = Converter.keys()
print(my_wfrs)

out_n = "This is an output file of the Hi-C processing pipeline"
int_n = "This is an intermediate file in the HiC processing pipeline"

success = 0
patch_problem = 0
deleted = 0
count = 0

for a_file in all_files:  
    count += 1
    my_wfr = a_file.get('workflow_run_outputs')
    
    if count % 30 == 0:
        print(count,success)
        
    if not my_wfr:
        continue
        
    wfr_title = my_wfr[0]['display_title'].split(' run')[0]
    
    if wfr_title in my_wfrs: 
        form = a_file['file_format']
        file_id = a_file['uuid']
        f_type =  Converter[wfr_title][form]
        desc = ""
        if f_type == "intermediate file":
            desc = int_n
        else:
            desc = out_n
        patch_data = {"description": desc, 'file_type': f_type}
        ff_utils.patch_metadata(patch_data, obj_id=file_id , key = my_key)
        success += 1
           
print(success)

